# How to stream events from a tool

If you have tools that call chat models, retrievers, or other runnables, you may want to access internal events from those runnables or configure them with additional properties. This guide shows you how to manually pass parameters properly so that you can do this using the astream_events() method.

Say you have a custom tool that calls a chain that condenses its input by prompting a chat model to return only 10 words, then reversing the output. First, define it in a naive way:

# Implementing a Streaming Tool
1. Naive Implementation

Define a summarization tool:

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.tools import tool

model = ChatOpenAI(model="gpt-4o-mini")

@tool
async def special_summarization_tool(long_text: str) -> str:
    """Summarizes input text using advanced techniques."""
    prompt = ChatPromptTemplate.from_template(
        "You are an expert writer. Summarize the following text in 10 words or less:\n\n{long_text}"
    )
    
    def reverse(x: str):
        return x[::-1]

    chain = prompt | model | StrOutputParser() | reverse
    return await chain.ainvoke({"long_text": long_text})


# 2. Fixing Config Propagation
Modify the tool to accept and propagate RunnableConfig:

In [2]:
from langchain_core.runnables import RunnableConfig

@tool
async def special_summarization_tool_with_config(
    long_text: str, config: RunnableConfig
) -> str:
    """Summarizes input text using advanced techniques."""
    prompt = ChatPromptTemplate.from_template(
        "You are an expert writer. Summarize the following text in 10 words or less:\n\n{long_text}"
    )

    def reverse(x: str):
        return x[::-1]

    chain = prompt | model | StrOutputParser() | reverse
    # Pass the config object to runnables
    return await chain.ainvoke({"long_text": long_text}, config=config)


# 3. Streaming Events
Stream the events using astream_events():

In [3]:
LONG_TEXT = """
NARRATOR:
(Black screen with text; The sound of buzzing bees can be heard)
...
"""

stream = special_summarization_tool_with_config.astream_events(
    {"long_text": LONG_TEXT}, version="v2"
)

async for event in stream:
    if event["event"] == "on_chat_model_end":
        print(event)


{'event': 'on_chat_model_end', 'data': {'output': AIMessage(content='Buzzing bees create atmosphere on black screen.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0705bf87c0'}, id='run-766eeafb-6765-49dd-ba1c-123d8719429b'), 'input': {'messages': [[HumanMessage(content='You are an expert writer. Summarize the following text in 10 words or less:\n\n\nNARRATOR:\n(Black screen with text; The sound of buzzing bees can be heard)\n...\n', additional_kwargs={}, response_metadata={})]]}}, 'run_id': '766eeafb-6765-49dd-ba1c-123d8719429b', 'name': 'ChatOpenAI', 'tags': ['seq:step:2'], 'metadata': {'ls_provider': 'openai', 'ls_model_name': 'gpt-4o-mini', 'ls_model_type': 'chat', 'ls_temperature': 0.7}, 'parent_ids': ['4c9906c2-4c7a-412d-9d29-c4f670ea79ab']}


# 4. Streaming Token Chunks
To stream tokens as they are generated

In [4]:
stream = special_summarization_tool_with_config.astream_events(
    {"long_text": LONG_TEXT}, version="v2"
)

async for event in stream:
    if event["event"] == "on_chat_model_stream":
        print(event)


{'event': 'on_chat_model_stream', 'data': {'chunk': AIMessageChunk(content='', additional_kwargs={}, response_metadata={}, id='run-41894b3d-bbaf-4c11-8543-633dac044a69')}, 'run_id': '41894b3d-bbaf-4c11-8543-633dac044a69', 'name': 'ChatOpenAI', 'tags': ['seq:step:2'], 'metadata': {'ls_provider': 'openai', 'ls_model_name': 'gpt-4o-mini', 'ls_model_type': 'chat', 'ls_temperature': 0.7}, 'parent_ids': ['54366a90-ffc2-45d4-a0d6-2d8749f06479']}
{'event': 'on_chat_model_stream', 'data': {'chunk': AIMessageChunk(content='Black', additional_kwargs={}, response_metadata={}, id='run-41894b3d-bbaf-4c11-8543-633dac044a69')}, 'run_id': '41894b3d-bbaf-4c11-8543-633dac044a69', 'name': 'ChatOpenAI', 'tags': ['seq:step:2'], 'metadata': {'ls_provider': 'openai', 'ls_model_name': 'gpt-4o-mini', 'ls_model_type': 'chat', 'ls_temperature': 0.7}, 'parent_ids': ['54366a90-ffc2-45d4-a0d6-2d8749f06479']}
{'event': 'on_chat_model_stream', 'data': {'chunk': AIMessageChunk(content=' screen', additional_kwargs={}, r

# Key Notes
* Manual Config Propagation: Always pass RunnableConfig for Python <=3.10.
* Event Types:
    * on_chat_model_end: Emits final results.
    * on_chat_model_stream: Emits intermediate tokens.